# Import packages

In [1]:
input_folder_path = r'C:\Users\cliff\OneDrive\Documents\Data Practicum1\CDP_Data'  
output_folder_path = r'C:\Users\cliff\OneDrive\Documents\Data Practicum1\CDP_Data\Corporate_Data'  
import os, glob, re, gc, warnings, pandas as pd, numpy as np; from concurrent.futures import ThreadPoolExecutor, as_completed; warnings.filterwarnings("ignore")

pd.set_option('display.max_columns', None)

In [84]:
!pip install squarify

# Get Emission data headers for each worksheet

As the CDP workbooks are quite large and difficult to work with this was an iterative process to work through all the worksheets and header names.  

In [7]:


def list_excel_headers_by_search_terms(folder_path, search_terms):
    """
    Index through Excel files in the specified folder (non-recursive) and list all of the column headers
    in any worksheet whose name contains any of the given search terms.

    For each Excel file found:
    - The function checks for any worksheet whose name contains any of the search_terms.
    - If found, it reads only the header row of that worksheet.
      For files from 2022 or 2023 (determined from the file name), it uses the second row (header=1);
      otherwise, it uses the first row (header=0).
    - It prints the file name, the matching worksheet name, and its column headers.
    - Files that do not contain a worksheet matching any of the search terms or that encounter an error are noted.

    Args:
        folder_path (str): The path to the folder containing the Excel files.
        search_terms (list of str): A list of substrings to search for in the worksheet names 
                                    (e.g., ["14.1", "CC14.1", "C6.5"]).

    Returns:
        dict: A dictionary where each key is a file name and its value is a tuple 
              (matching_worksheet, list of column headers).
    """
    # Find all Excel files in the folder (non-recursive)
    excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))
    headers_dict = {}

    def process_file(file):
        file_name = os.path.basename(file)
        # Extract year from file name (assumes a 4-digit year exists)
        year_match = re.search(r'(\d{4})', file_name)
        year = year_match.group(1) if year_match else None

        # Determine which row to use as header: second row for 2022 and 2023, otherwise first row.
        header_row = 1 if year in ["2022", "2023"] else 0

        try:
            # Create an ExcelFile object to check available sheets
            xls = pd.ExcelFile(file)
            # Look for a sheet that contains any of the search terms
            matching_sheet = next(
                (sheet for sheet in xls.sheet_names if any(term in sheet for term in search_terms)),
                None
            )
            if matching_sheet:
                # Read only the header row by specifying nrows=0 and the appropriate header row
                df = pd.read_excel(xls, sheet_name=matching_sheet, nrows=0, header=header_row)
                return file_name, matching_sheet, list(df.columns)
            else:
                print(f"No worksheet containing any of {search_terms} found in {file_name}")
        except Exception as e:
            print(f"Error processing {file_name}: {e}")
        return file_name, None, None

    # Use ThreadPoolExecutor for concurrent processing
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_file, file) for file in excel_files]
        for future in as_completed(futures):
            file_name, matching_sheet, headers = future.result()
            if headers is not None:
                headers_dict[file_name] = (matching_sheet, headers)

    # Print out the matching worksheet and column headers for each file
    for file_name, (matching_sheet, headers) in headers_dict.items():
        print(f"File: {file_name}")
        print(f"Worksheet: {matching_sheet}")
        print("Column Headers:", headers)
        print("-" * 40)

    return headers_dict



search_terms = ["8. Emissions - Data SME", "CC8. Emissions Data SME", "C6.1", "C6.3"]
headers_dict = list_excel_headers_by_search_terms(input_folder_path, search_terms)





NameError: name 'input_folder_path' is not defined

In [27]:


def consolidate_workbook_by_year(folder_path, search_terms, output_file):
    """
    Index through Excel files in the specified folder (non-recursive), search for any worksheet
    whose name contains any of the provided search_terms, and copy the entire worksheet data 
    (using the appropriate header row) to a new workbook. The header row is determined based on 
    the year extracted from the file name: for files from 2022 or 2023, the header is taken from 
    the second row; otherwise, from the first row.

    Data from files with the same year are concatenated together. In the new workbook, each sheet 
    is named after the year of the files it came from.

    Args:
        folder_path (str): The path to the folder containing the Excel files.
        search_terms (list of str): A list of substrings to search for in the worksheet names 
                                    (e.g., ["14.1", "CC14.1", "C6.5"]).
        output_file (str): The file path where the new consolidated workbook will be saved.

    Returns:
        None
    """
    # Find all Excel files in the folder (non-recursive)
    excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))
    # Dictionary to hold dataframes keyed by year
    data_by_year = {}

    def process_file(file):
        file_name = os.path.basename(file)
        # Extract the year from the file name (assumes a 4-digit year exists)
        year_match = re.search(r'(\d{4})', file_name)
        year = year_match.group(1) if year_match else "Unknown"
        # For files from 2022 or 2023, header is on the second row; otherwise, first row.
        header_row = 1 if year in ["2022", "2023"] else 0

        try:
            xls = pd.ExcelFile(file)
            # Find the first worksheet whose name contains any of the search_terms
            matching_sheet = next(
                (sheet for sheet in xls.sheet_names if any(term in sheet for term in search_terms)),
                None
            )
            if matching_sheet:
                # Read the entire sheet using the appropriate header row.
                df = pd.read_excel(xls, sheet_name=matching_sheet, header=header_row)
                return year, df
            else:
                print(f"No worksheet containing any of {search_terms} found in {file_name}")
        except Exception as e:
            print(f"Error processing {file_name}: {e}")
        return None, None

    # Process files concurrently
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_file, file) for file in excel_files]
        for future in as_completed(futures):
            year, df = future.result()
            if df is not None:
                if year in data_by_year:
                    data_by_year[year] = pd.concat([data_by_year[year], df], ignore_index=True)
                else:
                    data_by_year[year] = df

    # Write the consolidated dataframes to a new workbook, one sheet per year.
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        for year, df in data_by_year.items():
            # Ensure the sheet name is at most 31 characters
            sheet_name = str(year)[:31]
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"Consolidated workbook saved to: {output_file}")


search_terms = ["14.1", "CC14.1", "C6.5"]
output_file_path = r'C:\Users\cliff\OneDrive\Documents\Data Practicum1\CDP_Data\Corporate_Data\merged_scope3_worksheets.xlsx'  
consolidate_workbook_by_year(input_folder_path, search_terms, output_file_path)


Consolidated workbook saved to: C:\Users\cliff\OneDrive\Documents\Data Practicum1\CDP_Data\Corporate_Data\merged_scope3_worksheets.xlsx


In [ ]:


def create_CDP_summary_data_workbook(folder_path):
    """
    Index through Excel files in the specified folder (non-recursive) and list all of the column headers
    in the 'Summary Data' worksheet for each file.

    For each Excel file found:
    - If the 'Summary Data' worksheet exists, the function reads only the header (first row) of the worksheet.
    - It prints the file name along with its column headers.
    - Files that do not contain the 'Summary Data' sheet or encounter an error are noted.

    Args:
        folder_path (str): The path to the folder containing the Excel files.

    Returns:
        dict: A dictionary where each key is a file name and its value is a list of column headers.
    """
    # Find all Excel files in the folder (non-recursive)
    excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))
    headers_dict = {}

    def process_file(file):
        file_name = os.path.basename(file)
        try:
            # Create an ExcelFile object to check for available sheets
            xls = pd.ExcelFile(file)
            if 'Summary Data' in xls.sheet_names:
                # Read only the header row by specifying nrows=0
                df = pd.read_excel(xls, sheet_name='Summary Data', nrows=0)
                return file_name, list(df.columns)
            else:
                print(f"Worksheet 'Summary Data' not found in {file_name}")
        except Exception as e:
            print(f"Error processing {file_name}: {e}")
        return file_name, None

    # Use ThreadPoolExecutor for concurrent processing
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_file, file) for file in excel_files]
        for future in as_completed(futures):
            file_name, headers = future.result()
            if headers is not None:
                headers_dict[file_name] = headers

    # Print out the column headers for each file
    for file_name, headers in headers_dict.items():
        print(f"File: {file_name}")
        print("Column Headers:", headers)
        print("-" * 40)

    return headers_dict

headers_dict =  create_CDP_summary_data_workbook(folder_path)


In [7]:


def consolidate_CDP_worksheets(input_folder, output_folder, worksheets):
    """
    Index through Excel files in the specified input folder (non-recursive), read the worksheets 
    from each file that match the provided list (if they exist), and store them as dataframes. 
    For each (year, worksheet) pair, if multiple files are from the same year, their dataframes 
    are concatenated. Then, save all collected dataframes in a new Excel workbook where each 
    dataframe is saved as a separate worksheet named with the year and worksheet name.
    
    Args:
        input_folder (str): The path to the folder containing the Excel files.
        output_folder (str): The path to the folder where the combined workbook will be saved.
        worksheets (list of str): A list of worksheet names to search for and extract from each file.
        
    Returns:
        None
    """
    # Find all Excel files in the input folder (non-recursive)
    excel_files = glob.glob(os.path.join(input_folder, '*.xlsx'))
    # Dictionary with keys as (year, sheet_name) and values as the concatenated DataFrame
    dataframes_dict = {}

    def process_file(file):
        file_name = os.path.basename(file)
        # Attempt to extract a four-digit year from the file name (e.g., '2021')
        year_match = re.search(r'(\d{4})', file_name)
        year = year_match.group(1) if year_match else "Unknown"
        results = []
        try:
            xls = pd.ExcelFile(file)
            for sheet in worksheets:
                if sheet in xls.sheet_names:
                    df = pd.read_excel(xls, sheet_name=sheet)
                    results.append((year, sheet, df))
                else:
                    print(f"Worksheet '{sheet}' not found in {file_name}")
        except Exception as e:
            print(f"Error processing {file_name}: {e}")
        return results

    # Process files concurrently
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_file, file) for file in excel_files]
        for future in as_completed(futures):
            file_results = future.result()
            for year, sheet, df in file_results:
                key = (year, sheet)
                if key in dataframes_dict:
                    dataframes_dict[key] = pd.concat([dataframes_dict[key], df], ignore_index=True)
                else:
                    dataframes_dict[key] = df

    # Define the output Excel file path (saved in the output folder)
    output_file = os.path.join(output_folder, "combined_summary.xlsx")
    
    # Write all dataframes to a single workbook, each in its own sheet named "year_sheet"
    with pd.ExcelWriter(output_file, engine='openpyxl') as writer:
        for (year, sheet), df in dataframes_dict.items():
            # Create a sheet name combining year and sheet, ensuring it is a string and valid (max 31 characters)
            sheet_name = f"{year}_{sheet}"[:31]
            df.to_excel(writer, sheet_name=sheet_name, index=False)
    
    print(f"Combined workbook saved to: {output_file}")


worksheets = ["Summary Data", "Additional Data"]

consolidate_CDP_worksheets(input_folder, output_folder, ["14.1"])


NameError: name 'input_folder' is not defined

In [ ]:

    # Helper function to merge columns: takes the first non-null value among the candidate columns
    def merge_columns(df, new_col, candidates):
        merged = pd.Series([pd.NA] * len(df))
        for col in candidates:
            if col in df.columns:
                merged = merged.fillna(df[col])
        # Ensure the merged column is a string
        df[new_col] = merged.astype(str)
        # Drop the original candidate columns if they exist
        df.drop(columns=[col for col in candidates if col in df.columns], inplace=True)
        return df

    # Merge the specified columns into unified columns
    final_df = merge_columns(final_df, 'Account Numbers', ['account_id', 'Account number'])
    final_df = merge_columns(final_df, 'Countries', ['incorporated_country', 'Country', 'Country/Area'])
    final_df = merge_columns(final_df, 'Company Name', ['Organization', 'account_name'])
    final_df = merge_columns(final_df, 'Primary Stock Ticker', ['ticker', 'Primary Ticker','Tickers']) #these contain the primary stock tickers while 'tickers' is a list of all ticker possibilities
    final_df = merge_columns(final_df, 'Account Numbers', ['row', 'Row'])
    final_df = merge_columns(final_df, 'Scope 3 Sources', ['14.1 C1 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Sources of Scope 3 emissions',
                                                           'CC14.1 C1 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Sources of Scope 3 emissions',
                                                          'RowName'])
    final_df = merge_columns(final_df, 'Evaluation Status', ['14.1 C2 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Evaluation status',
                                                             'CC14.1 C2 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Evaluation status',
                                                            'C6.5_C1_Account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions. - Evaluation status',
                                                            'C6.5_C1_Account for your organization’s gross global Scope 3 emissions, disclosing and explaining any exclusions. - Evaluation status'])
    final_df = merge_columns(final_df, 'Scope 3 Amount', ['14.1 C3 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - metric tonnes CO2e',
                                                          'CC14.1 C3 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - metric tonnes CO2e',
                                                         'C6.5_C2_Account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions. - Metric tonnes CO2e',
                                                         'C6.5_C2_Account for your organization’s gross global Scope 3 emissions, disclosing and explaining any exclusions. - Metric tonnes CO2e',
                                                         'C6.5_C2_Account for your organization’s gross global Scope 3 emissions, disclosing and explaining any exclusions. - Emissions in reporting year (metric tons CO2e)'])
    final_df = merge_columns(final_df, 'Calculation Methodology', ['14.1 C4 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Methodology',
                                                                   'CC14.1 C4 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Emissions calculation methodology',
                                                                  'C6.5_C3_Account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions. - Emissions calculation methodology',
                                                                  'C6.5_C3_Account for your organization’s gross global Scope 3 emissions, disclosing and explaining any exclusions. - Emissions calculation methodology'])

In [3]:


def merge_columns(df, new_col, candidates, force_str=False):
    """
    Merges specified columns into a single column by taking the first non-null value.
    Drops the original columns once merged.
    """
    if not df.empty:
        # Initialize merged column as dtype=object to avoid implicit downcasting
        merged = pd.Series(pd.NA, dtype="object", index=df.index)

        for col in candidates:
            if col in df.columns:
                merged = merged.combine_first(df[col])  # Use .combine_first() to avoid .fillna()

        df[new_col] = merged
        
        # Convert to string only if necessary
        if force_str:
            df[new_col] = df[new_col].astype(str).replace("<NA>", pd.NA)  # Prevent '<NA>' artifacts
        
        df.drop(columns=[col for col in candidates if col in df.columns], inplace=True, errors='ignore')
    return df

def extract_and_merge_data(folder_path, search_terms):
    """
    Extracts data from Excel files in the specified folder, searching for worksheets that contain 
    any of the provided search_terms in their name. Uses different header rows based on the year in 
    the filename and merges extracted data into a single DataFrame.

    Args:
        folder_path (str): Path to the folder containing Excel files.
        search_terms (list of str): List of worksheet name substrings to search for.

    Returns:
        pd.DataFrame: A consolidated DataFrame containing data from all matching worksheets.
    """
    # Find all Excel files in the folder (non-recursive)
    excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))
    all_data = []  # List to store extracted DataFrames

    def process_file(file):
        file_name = os.path.basename(file)
        # Extract the year from the file name (assumes a 4-digit year exists)
        year_match = re.search(r'(\d{4})', file_name)
        year = year_match.group(1) if year_match else "Unknown"
        # For files from 2022 or 2023, header is on the second row; otherwise, first row.
        header_row = 1 if year in ["2022", "2023"] else 0

        try:
            xls = pd.ExcelFile(file)
            # Find the first worksheet whose name contains any of the search_terms
            matching_sheet = next(
                (sheet for sheet in xls.sheet_names if any(term in sheet for term in search_terms)),
                None
            )
            if matching_sheet:
                # Read the entire sheet using the appropriate header row
                df = pd.read_excel(xls, sheet_name=matching_sheet, header=header_row)
                df['Source File'] = file_name  # Track source file for reference
                df['Year'] = year  # Add extracted year column
                df['Year of Data'] = year  # Duplicate column for clarity
                return df
            else:
                print(f"No worksheet containing any of {search_terms} found in {file_name}")
        except Exception as e:
            print(f"Error processing {file_name}: {e}")
        return None

    # Process files concurrently
    with ThreadPoolExecutor() as executor:
        futures = [executor.submit(process_file, file) for file in excel_files]
        for future in as_completed(futures):
            df = future.result()
            if df is not None:
                all_data.append(df)

    # Concatenate all collected data
    if all_data:
        final_df = pd.concat(all_data, ignore_index=True)
        
        # Merge columns using the defined logic
        final_df = merge_columns(final_df, 'Account Numbers', ['account_id', 'Account number'], force_str=True)
        final_df = merge_columns(final_df, 'Countries', ['incorporated_country', 'Country', 'Country/Area'])
        final_df = merge_columns(final_df, 'Company Name', ['Organization', 'account_name'])
        final_df = merge_columns(final_df, 'Primary Stock Ticker', ['ticker', 'Primary Ticker','Tickers']) #these contain the primary stock tickers while 'tickers' is a list of all ticker possibilities
        final_df = merge_columns(final_df, 'Row_num', ['row', 'Row'])
        final_df = merge_columns(final_df, 'Scope 3 Sources', ['14.1 C1 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Sources of Scope 3 emissions',
                                                               'CC14.1 C1 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Sources of Scope 3 emissions',
                                                              'RowName'])
        final_df = merge_columns(final_df, 'Evaluation Status', ['14.1 C2 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Evaluation status',
                                                                 'CC14.1 C2 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Evaluation status',
                                                                'C6.5_C1_Account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions. - Evaluation status',
                                                                'C6.5_C1_Account for your organization’s gross global Scope 3 emissions, disclosing and explaining any exclusions. - Evaluation status'])
        final_df = merge_columns(final_df, 'Scope 3 Amount', ['14.1 C3 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - metric tonnes CO2e',
                                                              'CC14.1 C3 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - metric tonnes CO2e',
                                                             'C6.5_C2_Account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions. - Metric tonnes CO2e',
                                                             'C6.5_C2_Account for your organization’s gross global Scope 3 emissions, disclosing and explaining any exclusions. - Metric tonnes CO2e',
                                                             'C6.5_C2_Account for your organization’s gross global Scope 3 emissions, disclosing and explaining any exclusions. - Emissions in reporting year (metric tons CO2e)'])
        final_df = merge_columns(final_df, 'Calculation Methodology', ['14.1 C4 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Methodology',
                                                                       'CC14.1 C4 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Emissions calculation methodology',
                                                                      'C6.5_C3_Account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions. - Emissions calculation methodology',
                                                                      'C6.5_C3_Account for your organization’s gross global Scope 3 emissions, disclosing and explaining any exclusions. - Emissions calculation methodology'])
        final_df = merge_columns(final_df, 'Perc emissions calculated w primary data ', ['14.1 C5 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Percentage of emissions calculated using primary data',
                                                                                          'CC14.1 C5 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Percentage of emissions calculated using primary data',
                                                                                         'CC14.1 C5 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Percentage of emissions calculated using data obtained from suppliers or value chain partners',
                                                                                         'C6.5_C4_Account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions. - Percentage of emissions calculated using data obtained from suppliers or value chain partners',
                                                                                         'C6.5_C4_Account for your organization’s gross global Scope 3 emissions, disclosing and explaining any exclusions. - Percentage of emissions calculated using data obtained from suppliers or value chain partners'])
        final_df = merge_columns(final_df, 'Explanation', ['14.1 C6 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Explanation',
                                                           'CC14.1 C6 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Explanation',
                                                          'C6.5_C5_Account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions. - Explanation',
                                                          'C6.5_C5_Account for your organization’s gross global Scope 3 emissions, disclosing and explaining any exclusions. - Please explain'])

        final_df['Account Numbers'] = final_df['Account Numbers'].astype(str).str.replace(r'\.0$', '', regex=True)
        columns_to_drop = ['program_name', 'project_year', 'accounting_year', 'Source File', 'Year', 'Authority types', 'Primary questionnaire sector', 'Request response status', 'Access type', 'Samples', 'Response received date',
                   'Activities', 'Sectors', 'Industries', 'Primary ISIN', 'Access Types']  #dropping redundant columns in last two years of data

        # # Drop columns efficiently
        existing_cols_to_drop = [col for col in columns_to_drop if col in final_df.columns]
        final_df.drop(columns=existing_cols_to_drop, inplace=True, errors='ignore')
    
        # Force garbage collection to free memory
        
        gc.collect()

        return final_df
    else:
        print("No data extracted from the provided files.")
        return pd.DataFrame()  # Return an empty DataFrame if no data was found


search_terms = ["14.1", "CC14.1", "C6.5"]
final_df = extract_and_merge_data(input_folder_path, search_terms)




# Establish Final dataframe header names

In [20]:
final_df['Account Numbers'] = final_df['Account Numbers'].astype(str).str.replace(r'\.0$', '', regex=True)
columns_to_drop = ['program_name', 'project_year', 'accounting_year', 'Source File', 'Year', 'Authority types', 'Primary questionnaire sector', 'Request response status', 'Access type', 'Samples', 'Response received date',
                   'Activities', 'Sectors', 'Industries', 'Primary ISIN', 'Access Types']  #dropping redundant columns in last two years of data

# # Drop columns efficiently
existing_cols_to_drop = [col for col in columns_to_drop if col in final_df.columns]
final_df.drop(columns=existing_cols_to_drop, inplace=True, errors='ignore')

In [22]:
final_df.head()

,Year of Data,Primary activity,Primary sector,Primary industry,Account Numbers,Countries,Company Name,Primary Stock Ticker,Scope 3 Sources,Evaluation Status,Scope 3 Amount,Calculation Methodology,Perc emissions calculated w primary data,Explanation
0,2013,NaN,NaN,NaN,44,United Kingdom,3i Group,III LN,Purchased goods and services,Not evaluated,NaN,NaN,NaN,We do not have access to this data at this poi...
1,2013,NaN,NaN,NaN,44,United Kingdom,3i Group,III LN,Capital goods,Not evaluated,NaN,NaN,NaN,We do not have access to this data at this poi...
2,2013,NaN,NaN,NaN,44,United Kingdom,3i Group,III LN,Fuel-and-energy-related activities (not includ...,"Not relevant, explanation provided",NaN,NaN,NaN,We do not have any fuel or energy related acti...
3,2013,NaN,NaN,NaN,44,United Kingdom,3i Group,III LN,Upstream transportation and distribution,"Not relevant, explanation provided",NaN,NaN,NaN,NaN
4,2013,NaN,NaN,NaN,44,United Kingdom,3i Group,III LN,Waste generated in operations,"Relevant, calculated",12.34,Defra 2012,NaN,NaN


In [3]:

import os
import glob
import re
import pandas as pd
import gc
from concurrent.futures import ThreadPoolExecutor, as_completed

def merge_columns(df, new_col, candidates, force_str=False):
    """
    Merges specified columns into a single column by taking the first non-null value.
    Drops the original columns once merged.
    """
    if not df.empty:
        merged = pd.Series(pd.NA, dtype="object", index=df.index)
        for col in candidates:
            if col in df.columns:
                merged = merged.combine_first(df[col])

        df[new_col] = merged
        
        # Convert to string only if needed
        if force_str:
            df[new_col] = df[new_col].astype(str).replace("<NA>", pd.NA)
        
        # Drop original columns that were merged
        df.drop(columns=[col for col in candidates if col in df.columns], inplace=True, errors='ignore')
    return df

def process_file(file, search_terms):
    """
    Reads a single Excel file, finds a worksheet whose name contains any term in search_terms,
    extracts it (with a header adjustment based on year in filename), and returns a DataFrame.
    """
    file_name = os.path.basename(file)
    # Attempt to extract a 4-digit year
    year_match = re.search(r'(\d{4})', file_name)
    year = year_match.group(1) if year_match else "Unknown"
    
    # Adjust header row based on year
    header_row = 1 if year in ["2022", "2023"] else 0

    try:
        xls = pd.ExcelFile(file)
        # Find the first matching sheet
        matching_sheet = next(
            (sheet for sheet in xls.sheet_names if any(term in sheet for term in search_terms)),
            None
        )
        if matching_sheet:
            df = pd.read_excel(xls, sheet_name=matching_sheet, header=header_row)
            df['Source File'] = file_name
            df['Year'] = year
            df['Year of Data'] = year
            return df
        else:
            print(f"No matching worksheet found in: {file_name}")
    except Exception as e:
        print(f"Error processing {file_name}: {e}")
    
    return None

def merge_chunk_data(chunk_data):
    """
    Given a list of DataFrames from one chunk, concatenate them and merge key columns.
    Returns the chunk's merged DataFrame.
    """
    if not chunk_data:
        return pd.DataFrame()

    chunk_df = pd.concat(chunk_data, ignore_index=True)
    
    # Perform merges for columns of interest
    chunk_df = merge_columns(chunk_df, 'Account Numbers', ['account_id', 'Account number'], force_str=True)
    chunk_df = merge_columns(chunk_df, 'Countries', ['incorporated_country', 'Country', 'Country/Area'])
    chunk_df = merge_columns(chunk_df, 'Company Name', ['Organization', 'account_name'])
    chunk_df = merge_columns(chunk_df, 'Primary Stock Ticker', ['ticker', 'Primary Ticker','Tickers'])
    chunk_df = merge_columns(chunk_df, 'Row_num', ['row', 'Row'])
    chunk_df = merge_columns(chunk_df, 'Scope 3 Sources', [
        '14.1 C1 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Sources of Scope 3 emissions',
        'CC14.1 C1 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Sources of Scope 3 emissions',
        'RowName'])
    chunk_df = merge_columns(chunk_df, 'Evaluation Status', [
        '14.1 C2 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Evaluation status',
        'CC14.1 C2 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Evaluation status',
        'C6.5_C1_Account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions. - Evaluation status',
        'C6.5_C1_Account for your organization’s gross global Scope 3 emissions, disclosing and explaining any exclusions. - Evaluation status'])
    chunk_df = merge_columns(chunk_df, 'Scope 3 Amount', [
        '14.1 C3 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - metric tonnes CO2e',
        'CC14.1 C3 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - metric tonnes CO2e',
        'C6.5_C2_Account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions. - Metric tonnes CO2e',
        'C6.5_C2_Account for your organization’s gross global Scope 3 emissions, disclosing and explaining any exclusions. - Metric tonnes CO2e',
        'C6.5_C2_Account for your organization’s gross global Scope 3 emissions, disclosing and explaining any exclusions. - Emissions in reporting year (metric tons CO2e)'])
    chunk_df = merge_columns(chunk_df, 'Calculation Methodology', [
        '14.1 C4 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Methodology',
        'CC14.1 C4 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Emissions calculation methodology',
        'C6.5_C3_Account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions. - Emissions calculation methodology',
        'C6.5_C3_Account for your organization’s gross global Scope 3 emissions, disclosing and explaining any exclusions. - Emissions calculation methodology'])
    chunk_df = merge_columns(chunk_df, 'Perc emissions calculated w primary data ', [
        '14.1 C5 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Percentage of emissions calculated using primary data',
        'CC14.1 C5 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Percentage of emissions calculated using primary data',
        'CC14.1 C5 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Percentage of emissions calculated using data obtained from suppliers or value chain partners',
        'C6.5_C4_Account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions. - Percentage of emissions calculated using data obtained from suppliers or value chain partners',
        'C6.5_C4_Account for your organization’s gross global Scope 3 emissions, disclosing and explaining any exclusions. - Percentage of emissions calculated using data obtained from suppliers or value chain partners'])
    chunk_df = merge_columns(chunk_df, 'Explanation', [
        '14.1 C6 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Explanation',
        'CC14.1 C6 - Please account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions - Explanation',
        'C6.5_C5_Account for your organization’s Scope 3 emissions, disclosing and explaining any exclusions. - Explanation',
        'C6.5_C5_Account for your organization’s gross global Scope 3 emissions, disclosing and explaining any exclusions. - Please explain'])

    # Remove trailing ".0" from Account Numbers if present
    chunk_df['Account Numbers'] = (
        chunk_df['Account Numbers']
        .astype(str)
        .str.replace(r'\.0$', '', regex=True)
    )
    
    chunk_df['Row_num'] = (
        chunk_df['Row_num']
        .astype(str)
        .str.replace(r'\.0$', '', regex=True)
    )

    # Drop unwanted columns if they exist
    columns_to_drop = [
        'program_name', 'project_year', 'accounting_year', 'Source File', 'Year',
        'Authority types', 'Primary questionnaire sector', 'Request response status',
        'Access type', 'Samples', 'Response received date', 'Activities', 'Sectors',
        'Industries', 'Primary ISIN', 'Access Types'
    ]
    existing_cols = [c for c in columns_to_drop if c in chunk_df.columns]
    chunk_df.drop(columns=existing_cols, inplace=True, errors='ignore')

    # Define the desired order for the specified columns
    cols_order = ['Account Numbers', 'Year of Data', 'Company Name', 'Primary Stock Ticker', 'Countries', 
                 'Primary industry', 'Primary sector', 'Primary activity', 'Row_num','Scope 3 Sources', 'Scope 3 Amount', 'Evaluation Status', 'Calculation Methodology', 'Perc emissions calculated w primary data', 'Explanation']
    
    # Create a list of desired columns that exist in merged_df2
    existing_cols = [col for col in cols_order if col in final_df.columns]
    
    # Identify the remaining columns that aren't in the desired list
    remaining_cols = [col for col in final_df.columns if col not in existing_cols]

    # Run garbage collection
    gc.collect()

    return chunk_df

def extract_and_merge_data_in_batches(folder_path, search_terms, chunk_size=10):
    """
    Reads Excel files in smaller batches using concurrency to avoid memory spikes.
    Each batch is processed and merged into a single DataFrame, then appended 
    to a global list. The final DataFrame is concatenated from these chunks.
    """
    excel_files = glob.glob(os.path.join(folder_path, '*.xlsx'))
    
    # Store results for each chunk
    chunked_results = []

    # Batch processing in chunks
    for i in range(0, len(excel_files), chunk_size):
        file_batch = excel_files[i : i + chunk_size]
        
        # Step 1: Concurrently process each file in this batch
        batch_data = []
        with ThreadPoolExecutor() as executor:
            futures = [executor.submit(process_file, f, search_terms) for f in file_batch]
            for future in as_completed(futures):
                df = future.result()
                if df is not None:
                    batch_data.append(df)
        
        # Step 2: Merge the chunk’s data
        if batch_data:
            chunk_df = merge_chunk_data(batch_data)
            chunked_results.append(chunk_df)
        
        # Free memory
        del batch_data
        gc.collect()

    # Concatenate all chunked results
    if not chunked_results:
        print("No data extracted from any files.")
        return pd.DataFrame()

    final_df = pd.concat(chunked_results, ignore_index=True)
    return final_df


search_terms = ["14.1", "CC14.1", "C6.5"]
chunk_size = 10  # Number of files to process at once

final_df = extract_and_merge_data_in_batches(input_folder_path, search_terms, chunk_size=chunk_size)




In [18]:
# Define the desired order for the specified columns
cols_order = ['Account Numbers', 'Year of Data', 'Company Name', 'Primary Stock Ticker', 'Countries', 
             'Primary industry', 'Primary sector', 'Primary activity', 'Row_num','Scope 3 Sources', 'Scope 3 Amount', 'Evaluation Status', 'Calculation Methodology', 'Perc emissions calculated w primary data', 'Explanation']

# Create a list of desired columns that exist in merged_df2
existing_cols = [col for col in cols_order if col in final_df.columns]

# Identify the remaining columns that aren't in the desired list
remaining_cols = [col for col in final_df.columns if col not in existing_cols]



# Create a mask where 'Scope 3 Amount' is null AND 'Evaluation Status' is 'Not relevant, explanation provided'
mask = final_df['Scope 3 Amount'].isna() & (final_df['Evaluation Status'] == 'Not relevant, explanation provided')

# Assign 0 to 'Scope 3 Amount' where the mask is True
final_df.loc[mask, 'Scope 3 Amount'] = 0


In [20]:
final_df.head(50)

,Year of Data,Primary activity,Primary sector,Primary industry,Account Numbers,Countries,Company Name,Primary Stock Ticker,Row_num,Scope 3 Sources,Evaluation Status,Scope 3 Amount,Calculation Methodology,Perc emissions calculated w primary data,Explanation
0,2013,NaN,NaN,NaN,44,United Kingdom,3i Group,III LN,1,Purchased goods and services,Not evaluated,NaN,NaN,NaN,We do not have access to this data at this poi...
1,2013,NaN,NaN,NaN,44,United Kingdom,3i Group,III LN,2,Capital goods,Not evaluated,NaN,NaN,NaN,We do not have access to this data at this poi...
2,2013,NaN,NaN,NaN,44,United Kingdom,3i Group,III LN,3,Fuel-and-energy-related activities (not includ...,"Not relevant, explanation provided",0,NaN,NaN,We do not have any fuel or energy related acti...
3,2013,NaN,NaN,NaN,44,United Kingdom,3i Group,III LN,4,Upstream transportation and distribution,"Not relevant, explanation provided",0,NaN,NaN,NaN
4,2013,NaN,NaN,NaN,44,United Kingdom,3i Group,III LN,5,Waste generated in operations,"Relevant, calculated",12.34,Defra 2012,NaN,NaN
5,2013,NaN,NaN,NaN,44,United Kingdom,3i Group,III LN,6,Business travel,"Relevant, calculated",56.78,Defra 2012,NaN,NaN
6,2013,NaN,NaN,NaN,44,United Kingdom,3i Group,III LN,7,Employee commuting,"Not relevant, explanation provided",0,NaN,NaN,Our employees commute into central London usin...
7,2013,NaN,NaN,NaN,44,United Kingdom,3i Group,III LN,8,Upstream leased assets,Not evaluated,NaN,NaN,NaN,"We have not evaluated up-stream assets, and at..."
8,2013,NaN,NaN,NaN,44,United Kingdom,3i Group,III LN,9,Investments,"Not relevant, calculated",NaN,NaN,NaN,We plan to investigate what steps can be taken to
9,2013,NaN,NaN,NaN,44,United Kingdom,3i Group,III LN,10,Downstream transportation and distribution,"Not relevant, explanation provided",0,NaN,NaN,We are not involved in transoprt andf distribu...
